# Manufacturing Challenge 1 - Baseline Submission

This notebook provides a simple baseline for **Manufacturing Challenge 1: Predictive Maintenance**.

**Goal**: Predict `failure_24h` (binary) for each sensor reading
**Metric**: Macro-F1 Score - Higher is better
**Data**: Sensor readings (tabular only)

## Instructions:
1. **Replace API credentials** in the first cell with your team's API key and name
2. **Run all cells** to generate and submit baseline predictions
3. **Check the output** for your submission score

This baseline uses only tabular data with a simple Random Forest classifier.

In [ ]:
# 1. Initialize Client and Load Data

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from agentds import BenchmarkClient

# 🔑 REPLACE WITH YOUR CREDENTIALS
client = BenchmarkClient(
    api_key="your-api-key-here",        # Get from your team dashboard
    team_name="your-team-name-here"     # Your exact team name
)

# Load data from PVC paths
print("📂 Loading Manufacturing Challenge 1 data...")

# Load sensor readings data
train_sensors = pd.read_csv("/datasets/Manufacturing/sensor_readings_train.csv")
test_sensors = pd.read_csv("/datasets/Manufacturing/sensor_readings_test.csv")

print(f"✅ Data loaded:")
print(f"   Train sensors: {train_sensors.shape}")
print(f"   Test sensors: {test_sensors.shape}")
print(f"   Features: {list(train_sensors.columns)}")

In [ ]:
# 2. Tabular-Only Baseline Model and Predictions

# Select numeric sensor features for baseline
sensor_features = ['temperature_c', 'vibration_mms', 'pressure_kpa']
print(f"📊 Using sensor features: {sensor_features}")

# Prepare training data
X_train = train_sensors[sensor_features].fillna(0)  # Simple imputation
y_train = train_sensors['failure_24h']  # Binary target

# Prepare test data
X_test = test_sensors[sensor_features].fillna(0)

# Train simple Random Forest classifier baseline
print("🤖 Training Random Forest classifier...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)



# Create submission file
submission_df = pd.DataFrame({
    'timestamp': test_sensors['timestamp'],
    'machine_id': test_sensors['machine_id'],
    'failure_24h': predictions
})

# Save predictions
submission_df.to_csv("manufacturing_challenge1_predictions.csv", index=False)
print(f"✅ Predictions saved: {submission_df.shape[0]} predictions")
print(f"   Preview: {submission_df.head(3)}")
print(f"   Failure rate: {predictions.mean():.3f} ({predictions.sum()} failures out of {len(predictions)})")

In [ ]:
# 3. Submit Predictions

# Submit predictions to the competition
print("🚀 Submitting predictions...")

try:
    result = client.submit_prediction("Manufacturing", 1, "manufacturing_challenge1_predictions.csv")
    
    if result['success']:
        print("✅ Submission successful!")
        print(f"   📊 Score: {result['score']:.4f}")
        print(f"   📏 Metric: {result['metric_name']}")
        print(f"   ✔️  Validation: {'Passed' if result['validation_passed'] else 'Failed'}")
    else:
        print("❌ Submission failed!")
        print(f"   Error details: {result.get('details', {}).get('validation_errors', 'Unknown error')}")
        
except Exception as e:
    print(f"💥 Submission error: {e}")
    print("🔧 Check your API key and team name are correct!")

print("\n🎯 Next steps:")
print("   1. Try incorporating operator_logs.json text data")
print("   2. Engineer time-based features from timestamps")
print("   3. Move on to Manufacturing Challenge 2 (quality cost)!")